In [89]:
"""
Ipython notebook to test/document function that loads
the PRMS data file into a pandas dataframe
Author: John Volk, June 2016
Python 2.7.7
Pandas 0.18.1
"""
import numpy as np
import pandas as pd
import os

In [90]:
def load_data_file(data_file):
    """ 
    Read the data file and load into a datetime indexed Pandas dataframe object
    INPUT: data_file = data file path (string)
    OUTPUT: df = Pandas dataframe of input time series data from data file with datetime index
    """
    ## valid input time series that can be put into a data file
    valid_input_variables = ('gate_ht', 
                             'humidity', 
                             'lake_elev', 
                             'pan_evap',
                             'precip', 
                             'rain_day', 
                             'runoff', 
                             'snowdepth', 
                             'solrad', 
                             'tmax', 
                             'tmin', 
                             'wind_speed')
    #### starting list of names for header in dataframe
    date_header = ['year', 
                   'month', 
                   'day', 
                   'hh', 
                   'mm', 
                   'sec']
    ## append to header list the variables present in the file
    with open(data_file, 'r') as inf:
        for idx,l in enumerate(inf):
            if idx == 0: ## first line always string identifier of the file- may use later
                data_head = l.rstrip()
            elif l.startswith('/'): ## comment lines
                continue
            if l.startswith(valid_input_variables): ## header lines with name and number of input variables
                h = l.split() ## split line into list, first element name and second number of columns
                if int(h[1]) > 1: ## more than one input time series of a particular variable
                    for el in range(int(h[1])):
                        tmp = '{var_name} {var_ind}'.format(var_name=h[0], var_ind=el+1)
                        date_header.append(tmp)
                elif int(h[1]) == 1:
                    date_header.append(h[0])
            if l.startswith('#'): ## end of header info and begin time series input data
                skip_line = idx+1
                break
    ## read data file into pandas dataframe object with correct header names
    missing_value= -999 ## missing data representation
    df = pd.read_csv(data_file, header=-1, skiprows=skip_line, 
                     delim_whitespace=True, na_values=[missing_value]) ## read file
    df.columns = date_header  ## apply correct header names using metadata retrieved from file
    date = pd.Series(pd.to_datetime(df.year*10000+df.month*100+df.day, format='%Y%m%d'), 
                     index=df.index) ## create date column
    df.index = pd.to_datetime(date) ## make the df index the datetime 
    df.drop(['year', 'month', 'day', 'hh', 'mm', 'sec'], axis=1, inplace=True) ## drop unneeded columns
    df.columns.name = 'input variables' ; df.index.name = 'date' ## name dataframe axes (index,columns)
    return df

In [91]:
data_file = os.path.join(os.getcwd(),'..','models/lbcd/lbcd_2.data') 

In [92]:
load_data_file(data_file).head()

input variables,runoff 1,runoff 2,runoff 3,precip,tmax,tmin
date,,,,,,
1992-10-01,1.2,1.9,NaN,0.0,77,52.0
1992-10-02,1.3,2.0,NaN,0.0,75,53.0
1992-10-03,1.4,2.1,NaN,0.0,62,44.0
1992-10-04,1.5,2.2,NaN,0.0,62,34.0
1992-10-05,1.5,2.2,NaN,0.0,62,35.0


In [93]:
import time
def time_it(f, *args):
    start = time.clock()
    f(*args)
    return (time.clock() - start)
n = 50
t = 0
for i in range(n):
    t += time_it(load_data_file, data_file)
print 'The average time it took to run the function over {n} runs was {avg} seconds'.format(n=n,avg=t/n)

The average time it took to run the function over 50 runs was 0.005872 seconds


In [94]:
class Data(object):
    """
    PRMS data file object to read, write and modify time 
    series input to PRMS. Work in progress
    """
    
    def __init__(self, base_file):
        self.base_file = base_file

        
        

In [95]:
x=Data(data_file)

In [96]:
x.base_file

'/home/john/PRMS-Python/scripts/../models/lbcd/lbcd_2.data'

In [97]:
def get_datafile_metadata(data_file):
    """ 
    """
    ## valid input time series that can be put into a data file
    valid_input_variables = ('gate_ht', 
                             'humidity', 
                             'lake_elev', 
                             'pan_evap',
                             'precip', 
                             'rain_day', 
                             'runoff', 
                             'snowdepth', 
                             'solrad', 
                             'tmax', 
                             'tmin', 
                             'wind_speed')
    #### starting list of names for header in dataframe
    input_data_names = []
    ## append to header list the variables present in the file
    with open(data_file, 'r') as inf:
        for idx,l in enumerate(inf):
            if idx == 0: ## first line always string identifier of the file- may use later
                data_head = l.rstrip()
            elif l.startswith('/'): ## comment lines
                continue
            if l.startswith(valid_input_variables): ## header lines with name and number of input variables
                h = l.split() ## split line into list, first element name and second number of columns
                if int(h[1]) > 1: ## more than one input time series of a particular variable
                    for el in range(int(h[1])):
                        tmp = '{var_name} {var_ind}'.format(var_name=h[0], var_ind=el+1)
                        input_data_names.append(tmp)
                elif int(h[1]) == 1:
                    input_data_names.append(h[0])
            if l.startswith('#'): ## end of header info and begin time series input data
                data_startline = idx+1 ## 0 indexed line of first data entry
                break
                
    return dict([('data_startline',data_startline), ('data_variables',input_data_names)])

In [98]:
get_datafile_metadata(data_file)

{'data_startline': 18,
 'data_variables': ['runoff 1',
  'runoff 2',
  'runoff 3',
  'precip',
  'tmax',
  'tmin']}

In [99]:
def load_datafile(data_file):
    missing_value= -999 ## missing data representation
    df = pd.read_csv(data_file, header=-1, skiprows=get_data_metadata(data_file)['data_startline'], 
                     delim_whitespace=True, na_values=[missing_value]) ## read file
    #### starting list of names for header in dataframe
    date_header = ['year', 
                   'month', 
                   'day', 
                   'hh', 
                   'mm', 
                   'sec']
    df.columns = date_header + get_data_metadata(data_file)['data_variables']
    date = pd.Series(pd.to_datetime(df.year*10000+df.month*100+df.day, format='%Y%m%d'), 
                     index=df.index) ## create date column
    df.index = pd.to_datetime(date) ## make the df index the datetime 
    df.drop(['year', 'month', 'day', 'hh', 'mm', 'sec'], axis=1, inplace=True) ## drop unneeded columns
    df.columns.name = 'input variables' ; df.index.name = 'date' ## name dataframe axes (index,columns)
    return df

In [100]:
df = load_datafile(data_file)

In [101]:
def adjust_datafile(func, data_file, vars_to_adjust):
    df = load_datafile(data_file)
    for v in vars_to_adjust:
        df[v] = df[v].apply(func)
    return df

In [102]:
df.head()

input variables,runoff 1,runoff 2,runoff 3,precip,tmax,tmin
date,,,,,,
1992-10-01,1.2,1.9,NaN,0.0,77,52.0
1992-10-02,1.3,2.0,NaN,0.0,75,53.0
1992-10-03,1.4,2.1,NaN,0.0,62,44.0
1992-10-04,1.5,2.2,NaN,0.0,62,34.0
1992-10-05,1.5,2.2,NaN,0.0,62,35.0


In [103]:
def f(x):
    return np.sin(x)

In [104]:
adjust_datafile(f,data_file,['tmin','tmax']).head()

input variables,runoff 1,runoff 2,runoff 3,precip,tmax,tmin
date,,,,,,
1992-10-01,1.2,1.9,NaN,0.0,0.999520,0.986628
1992-10-02,1.3,2.0,NaN,0.0,-0.387782,0.395925
1992-10-03,1.4,2.1,NaN,0.0,-0.739181,0.017702
1992-10-04,1.5,2.2,NaN,0.0,-0.739181,0.529083
1992-10-05,1.5,2.2,NaN,0.0,-0.739181,-0.428183


In [67]:
df.head()

input variables,runoff 1,runoff 2,runoff 3,precip,tmax,tmin
date,,,,,,
1992-10-01,1.2,1.9,NaN,0.0,77,52.0
1992-10-02,1.3,2.0,NaN,0.0,75,53.0
1992-10-03,1.4,2.1,NaN,0.0,62,44.0
1992-10-04,1.5,2.2,NaN,0.0,62,34.0
1992-10-05,1.5,2.2,NaN,0.0,62,35.0


In [111]:
def write_datafile(data_file, df, out_name):
    ## reconstruct original datafile format
    date_header = ['year', 
                   'month', 
                   'day', 
                   'hh', 
                   'mm', 
                   'sec']
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['hh'] = df['mm'] = df['sec'] = 0
    df = df[date_header + get_data_metadata(data_file)['data_variables']] 
    with open(out_name,'w') as outf:
        with open(data_file) as data:
            for idx, line in enumerate(data):
                if idx == get_data_metadata(data_file)['data_startline']:
                    df.to_csv(outf, sep=' ', header=None, index=False, na_rep=-999)
                    break
                outf.write(line) # write line by line the header lines from original

In [112]:
## test loading, adjusting and writing at once
adj_df = adjust_datafile(f, data_file, ['tmax'])
write_datafile(data_file, adj_df, 'sine_tmax.data')